In [21]:
from fastembed import TextEmbedding
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import requests 
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct


## Q1. Embedding the query

In [3]:
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")
query = "I just discovered the course. Can I join now?"
embedding = next(model.embed([query]))
embedding_np = np.array(embedding)
min_value = np.min(embedding_np)
f"Minimum value in embedding: {min_value}"

Fetching 5 files: 100%|██████████| 5/5 [00:02<00:00,  2.01it/s]


'Minimum value in embedding: -0.11726373551188797'

In [5]:
embedding.dot(embedding)

np.float64(1.0)

## Q2. Cosine similarity with another vector

In [6]:
doc = 'Can I still join the course after the start date?'
embedding_doc = next(model.embed([doc]))
embedding.dot(embedding_doc)

np.float64(0.9008528856818037)

## Q3. Ranking by cosine

In [7]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [10]:
doc_embeddings = list(model.embed([doc['text'] for doc in documents]))


In [12]:
query_vector = embedding_np.reshape(1, -1)
cosine_similarities = cosine_similarity(query_vector, doc_embeddings)[0]

In [14]:
most_similar_index = int(np.argmax(cosine_similarities))
"Document index with highest similarity:", most_similar_index

('Document index with highest similarity:', 1)

## Q4. Ranking by cosine, version two

In [15]:
full_doc_embeddings = list(model.embed([doc['question'] + ' ' + doc['text'] for doc in documents]))

In [16]:
cosine_similarities = cosine_similarity(query_vector, full_doc_embeddings)[0]
most_similar_index = int(np.argmax(cosine_similarities))
"Document index with highest similarity:", most_similar_index

('Document index with highest similarity:', 0)

## Q5. Selecting the embedding model

What's the smallest dimensionality for models in fastembed? 384

In [17]:
model = TextEmbedding(model_name="BAAI/bge-small-en")

Fetching 5 files: 100%|██████████| 5/5 [00:03<00:00,  1.28it/s]


## Q6. Indexing with qdrant

In [19]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [24]:
texts = [doc['question'] + " " + doc['text'] for doc in documents]
full_doc_embeddings = list(model.embed(texts))

In [22]:
client = QdrantClient(":memory:")

In [23]:
client.recreate_collection(
    collection_name="course_docs",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)


/var/folders/ml/gp6bsjkn7w77hgytpxvg8gsw0000gn/T/ipykernel_63311/2935666450.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [25]:
points = [
    PointStruct(id=i, vector=embedding, payload={"text": texts[i]})
    for i, embedding in enumerate(full_doc_embeddings)
]
client.upsert(collection_name="course_docs", points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [29]:
query_vector.shape

(384,)

In [30]:
query_vector = next(model.embed([query]))
search_result = client.search(
    collection_name="course_docs",
    query_vector=query_vector,
    limit=5
)

/var/folders/ml/gp6bsjkn7w77hgytpxvg8gsw0000gn/T/ipykernel_63311/2662488397.py:2: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(


In [31]:
highest_score = max([hit.score for hit in search_result])
"Highest similarity score:", highest_score

('Highest similarity score:', 0.8703173398971558)